In [4]:
import pandas as pd

In [34]:
df = pd.read_csv("results.csv")
df[df['tournament'] == 'UEFA Euro']

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
4747,1960-07-06,Czechoslovakia,Russia,0,3,UEFA Euro,Marseille,France,True
4748,1960-07-06,France,Yugoslavia,4,5,UEFA Euro,Paris,France,False
4750,1960-07-09,France,Czechoslovakia,0,2,UEFA Euro,Marseille,France,False
4755,1960-07-10,Russia,Yugoslavia,2,1,UEFA Euro,Paris,France,True
5644,1964-06-17,Denmark,Russia,0,3,UEFA Euro,Barcelona,Spain,True
...,...,...,...,...,...,...,...,...,...
38168,2016-07-02,Germany,Italy,1,1,UEFA Euro,Bordeaux,France,True
38171,2016-07-03,France,Iceland,5,2,UEFA Euro,Saint-Denis,France,False
38173,2016-07-06,Portugal,Wales,2,0,UEFA Euro,Lyon,France,True
38174,2016-07-07,France,Germany,2,0,UEFA Euro,Marseille,France,False


In [30]:
df[df['tournament'] == 'UEFA Nations League']

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
39998,2018-09-06,Germany,France,0,0,UEFA Nations League,Munich,Germany,False
39999,2018-09-06,Czech Republic,Ukraine,1,2,UEFA Nations League,Uherské Hradiště,Czech Republic,False
40000,2018-09-06,Wales,Republic of Ireland,4,1,UEFA Nations League,Cardiff,Wales,False
40001,2018-09-06,Norway,Cyprus,2,0,UEFA Nations League,Oslo,Norway,False
40002,2018-09-06,Slovenia,Bulgaria,1,2,UEFA Nations League,Ljubljana,Slovenia,False
...,...,...,...,...,...,...,...,...,...
41881,2020-11-18,Georgia,Estonia,0,0,UEFA Nations League,Tbilisi,Georgia,False
41882,2020-11-18,Kosovo,Moldova,1,0,UEFA Nations League,Pristina,Kosovo,False
41883,2020-11-18,Greece,Slovenia,0,0,UEFA Nations League,Athens,Greece,False
41884,2020-11-18,Albania,Belarus,3,2,UEFA Nations League,Tirana,Albania,False
